## Tests

Test: check that the gb genebank file info matches a fafsa file sequence. 
A 'diff'.

python3 -m venv .venv-dependencies
source .venv-dependencies/bin/activate
pip install -r requirements.txt
python -m ipykernel install --user \
    --name dependencies-venv \
    --display-name "Python (.venv-dependencies)"


In [2]:
# Run this code inside a Jupyter cell. If all three lines point to your .venv folder in WSL, your setup is perfect:

import sys
import os

print(f"1. Executable: {sys.executable}") 
print(f"2. Version: {sys.version}")
print(f"3. Env Path: {os.getenv('VIRTUAL_ENV')}")

1. Executable: /home/morgan/projects/bioinfo-projects/comparative-sequence-analysis/.venv/bin/python
2. Version: 3.11.2 (main, Apr 28 2025, 14:11:48) [GCC 12.2.0]
3. Env Path: /home/morgan/projects/bioinfo-projects/comparative-sequence-analysis/.venv


In [27]:
# Basic imports. Probably overkill
from os import path
import Bio
from Bio import SeqIO
from Bio import GenBank
from Bio.Align import Alignment
import numpy as np
from Bio import Align

In [8]:

def get_gb_sequence__orig(g_path):
    with open(g_path, 'r') as G_file: 
        records = [x for  x in GenBank.parse(G_file)]
        if not len(records)==1:
           raise RuntimeError(f"Number of records not handled: {str(len(records))}")
        return records[0].sequence

def get_gb_sequence(g_path):
    for ind,seq_record in enumerate(SeqIO.parse(g_path, "gb")):
        print('--- seq number', ind)
        print(seq_record.id)
        print(repr(seq_record.seq))
        print(len(seq_record))
    return seq_record.seq

def get_fasta_sequence(f_path):
    for ind,seq_record in enumerate(SeqIO.parse(f_path, "fasta")):
        print('--- seq number', ind)
        print(seq_record.id)
        print(repr(seq_record.seq))
        print(len(seq_record))
    return seq_record.seq

def verify_gb_fafsa_compatibility(g, f, relpath=''):
    g_path = path.join(relpath, g)
    f_path = path.join(relpath, f)
    print('gb:')
    gb_seq = get_gb_sequence(g_path)
    print('fasta:')
    fa_seq = get_fasta_sequence(f_path)
    print('done')
    return gb_seq, fa_seq

In [9]:
gb_seq, fa_seq = verify_gb_fafsa_compatibility('sequence.gb', 'ncbi_dataset/data/gene.fna', relpath='./data/sirt1-human')

gb:
--- seq number 0
NM_001142498.2
Seq('GCATCTCCTCCTCCCTCTCCCCGGGCTCCTACTGGCCTGAGGTTGAGGGCGGCT...TTA')
3588
fasta:
--- seq number 0
NC_000010.11:67884656-67918390
Seq('GCCAGTGCCGCGCGTCGAGCGGGAGCAGAGGAGGCGAGGGAGGAGGGCCAGAGA...TTA')
33735
--- seq number 1
NC_060934.1:68753401-68787131
Seq('GCCAGTGCCGCGCGTCGAGCGGGAGCAGAGGAGGCGAGGGAGGAGGGCCAGAGA...TTA')
33731
done


In [ ]:
print(gb_seq[:120])
print(fa_seq[25:][:120])


GCATCTCCTC
CAGAGGAGGC


In [ ]:
def get_fasta_sequence_length(f_path):
    for ind,seq_record in enumerate(SeqIO.parse(f_path, "fasta")):
        return len(seq_record.seq)

files_found = []
for root, dirs, files in os.walk("data"):
    # print(f"Current Directory: {root}")
    # print(f"Subdirectories: {dirs}")
    # print(f"Files: {files}")
    # print("-" * 20)

    # see also https://biopython.org/docs/latest/Tutorial/chapter_seqio.html
    
    DEBUG=False
    for f in files:
        if f[-6:]=='.fasta':
            fpath = root.split('/') + dirs + [f]
            printing_str = '__'.join(fpath)
            if DEBUG and 'human' in root:
                print (fpath)
            try:
                with open('/'.join(fpath), 'r') as filetoread:
                    print (printing_str)
                    print('  ', str(get_fasta_sequence_length(filetoread)))
                files_found.append('/'.join(fpath))
            except:
                print("Could not open this file or commence sequence-length-getting operations on it:", printing_str)
    


data__sir2-p-aeruginosa__sequence(4).fasta
   250
data__sir2-yeast__sequence(3).fasta
   562
data__sirt1-horn-fly__sequence-sirtuin-1.fasta
   872
data__sirt1-horn-fly__sequence-sirtuin-2.fasta
   381
data__sirt1-mouse__sequence(1).fasta
   576
Could not open this file or commence sequence-length-getting operations on it: data__sirt1-human__ncbi_dataset__sequence(2).fasta


In [13]:
files_found

['data/sir2-p-aeruginosa/sequence(4).fasta',
 'data/sir2-yeast/sequence(3).fasta',
 'data/sirt1-horn-fly/sequence-sirtuin-1.fasta',
 'data/sirt1-horn-fly/sequence-sirtuin-2.fasta',
 'data/sirt1-mouse/sequence(1).fasta']

In [14]:
from Bio import SeqIO
sequences = [SeqIO.read(file_, 'fasta') for file_ in files_found]

In [21]:
for s in sequences:
    print(s, '\n', len(s.seq), '\n')

ID: XLJ82145.1
Name: XLJ82145.1
Description: XLJ82145.1 SIR2 family NAD-dependent protein deacylase [Pseudomonas aeruginosa]
Number of features: 0
Seq('MRAVVELLAGARRLVIFTGAGVSAESGIPTFRDALGGLWARYDPAALATPAAFA...FPG') 
 250 

ID: CAA96447.1
Name: CAA96447.1
Description: CAA96447.1 SIR2 [Saccharomyces cerevisiae]
Number of features: 0
Seq('MTIPHMKYAVSKTSENKVSNTVSPTQDKDAIRKQPDDIINNDEPSHKKIKVAQP...KTL') 
 562 

ID: XP_075153850.1
Name: XP_075153850.1
Description: XP_075153850.1 sirtuin 1 [Haematobia irritans]
Number of features: 0
Seq('MMDSSRQAVLSSERLKDIDDIHPIEFPEKADFDKFSVKTQNFTFGANILNTTTM...GPS') 
 872 

ID: XP_075148462.1
Name: XP_075148462.1
Description: XP_075148462.1 sirtuin 2 [Haematobia irritans]
Number of features: 0
Seq('MSESPTSSNKNKSKDDNETSSSAQNEEDNTIEVIRKFFTQKLNLVTSLDEEDGA...DAV') 
 381 

ID: AAI52315.1
Name: AAI52315.1
Description: AAI52315.1 Sirt1 protein [Mus musculus]
Number of features: 0
Seq('MAAAAAAAAIGYRGPYTFVQQHLMIGTDPRTILKDLLPETIPPPELDDMTLWQI...DKS') 
 576 



In [22]:
from Bio.Seq import Seq
seq1, seq2 = sequences[4].seq, sequences[1].seq # arbitrary

In [25]:
for s in sequences:
    print (s.id)

XLJ82145.1
CAA96447.1
XP_075153850.1
XP_075148462.1
AAI52315.1


In [43]:
Align.PairwiseAligner().align(sequences[0], sequences[1])

<PairwiseAlignments object (>9223372036854775807 alignments; score=-199) at 0x7f641377e710>

In [99]:
aligner = Align.PairwiseAligner(mode='local') # local is better than global or fogsaa in this task of long sequences.
def generate_alignments(this_aligner, printing=False):
    this_alignments = dict()
    if printing: print('generating alignments...')
    for i in range(5):
        if i==4: break
        for j in range(i+1,5):
            si, sj=sequences[i], sequences[j]
            alignment = this_aligner.align(si, sj)
            this_alignments[('rect', i, j)] = alignment
            if printing: print(i,j,si.id, sj.id, alignment.score )
    if printing: print()
    for i in range(5):
        if i==4: break
        for j in range(i+1,5):
            si, sj = sequences[i][1:], sequences[j]
            alignment = this_aligner.align(si, sj)
            this_alignments[('offset', i, j)] = alignment
            if printing: print(i,j,si.id, sj.id, alignment.score )
    return this_alignments

alignments = generate_alignments(aligner, printing=True)
# aligner.score(seq1, seq2)

generating alignments...
0 1 XLJ82145.1 CAA96447.1 48.0
0 2 XLJ82145.1 XP_075153850.1 53.0
0 3 XLJ82145.1 XP_075148462.1 56.0
0 4 XLJ82145.1 AAI52315.1 65.0
1 2 CAA96447.1 XP_075153850.1 149.0
1 3 CAA96447.1 XP_075148462.1 86.0
1 4 CAA96447.1 AAI52315.1 103.0
2 3 XP_075153850.1 XP_075148462.1 100.0
2 4 XP_075153850.1 AAI52315.1 228.0
3 4 XP_075148462.1 AAI52315.1 106.0

0 1 XLJ82145.1 CAA96447.1 48.0
0 2 XLJ82145.1 XP_075153850.1 53.0
0 3 XLJ82145.1 XP_075148462.1 56.0
0 4 XLJ82145.1 AAI52315.1 65.0
1 2 CAA96447.1 XP_075153850.1 149.0
1 3 CAA96447.1 XP_075148462.1 86.0
1 4 CAA96447.1 AAI52315.1 103.0
2 3 XP_075153850.1 XP_075148462.1 100.0
2 4 XP_075153850.1 AAI52315.1 228.0
3 4 XP_075148462.1 AAI52315.1 106.0


In [100]:
for si,s in enumerate(sequences):
    print(si,'',s.description)

0  XLJ82145.1 SIR2 family NAD-dependent protein deacylase [Pseudomonas aeruginosa]
1  CAA96447.1 SIR2 [Saccharomyces cerevisiae]
2  XP_075153850.1 sirtuin 1 [Haematobia irritans]
3  XP_075148462.1 sirtuin 2 [Haematobia irritans]
4  AAI52315.1 Sirt1 protein [Mus musculus]


In [ ]:
ex_aln = alignments['rect', 0,1]
for v in alignments.values():
    print(v)


<PairwiseAlignments object (>9223372036854775807 alignments; score=-199) at 0x7f64131065d0>
<PairwiseAlignments object (>9223372036854775807 alignments; score=-479) at 0x7f6413106750>
<PairwiseAlignments object (>9223372036854775807 alignments; score=-48) at 0x7f6412f35310>
<PairwiseAlignments object (>9223372036854775807 alignments; score=-209) at 0x7f6412f35e90>
<PairwiseAlignments object (>9223372036854775807 alignments; score=-118) at 0x7f6413153990>
<PairwiseAlignments object (>9223372036854775807 alignments; score=-53) at 0x7f6412f3f490>
<PairwiseAlignments object (>9223372036854775807 alignments; score=63) at 0x7f641314c350>
<PairwiseAlignments object (>9223372036854775807 alignments; score=-290) at 0x7f641314c9d0>
<PairwiseAlignments object (>9223372036854775807 alignments; score=13) at 0x7f641301b0d0>
<PairwiseAlignments object (>9223372036854775807 alignments; score=-43) at 0x7f641301b450>
<PairwiseAlignments object (>9223372036854775807 alignments; score=-201) at 0x7f641301b

Bio.Align.Alignment

In [95]:
alignments = generate_alignments(Align.PairwiseAligner(mode='local'))
ex_aln = alignments[('rect',1,4)]



In [128]:
test_aligner = Align.PairwiseAligner(mode='local')
test_aligner.open_gap_score = -10
test_aligner.extend_gap_score = -4.7
ex_aln = test_aligner.align(sequences[2], sequences[3])
ex_aln


<PairwiseAlignments object (3 alignments; score=61) at 0x7f641238bcd0>

In [130]:
for x in ex_aln:
    print(x)
    print('='*80)

XP_075153       183 PLNWVQKQIHSGVDPREILSKFLPPSIQRISPELTDMTLWRILASMMAEPPRRKKLSYVN
                  0 |.........................|..|....|.........................
XP_075148         4 PTSSNKNKSKDDNETSSSAQNEEDNTIEVIRKFFTQKLNLVTSLDEEDGANKKIFDNLTF

XP_075153       243 TFEDVIDLLKKSKNIIVLTGAGVSVSCGIPDFRSSDGIYSRLAKDFPNLPDPQAMFDINF
                 60 .........|..||.|...|||.|.|.||||||-|.|..........|||.|.|.|....
XP_075148        64 EGLCDHWKSKGFKNVITMVGAGISTSAGIPDFR-SPGSGLYDNLQKYNLPHPSAIFEMDY

XP_075153       303 FSKDPRPFYKFAREIYPGQFKPSPCHRFIKMLEQKQKLLRNYTQNIDTLEQVAGIKNVIE
                120 |...|.||...|.|.|||.|.|.|.|.|...|..|..|||.|||||||||..|||.....
XP_075148       123 FEENPQPFFALAKELYPGSFNPTPSHYFVRLLHEKGLLLRHYTQNIDTLERIAGIPADKL

XP_075153       363 CHGSFSTASCTKCKYKCDADTIRADIFAQRIPVCPQCQPNVEQSVDASQPVSEMELRMIV
                180 ................................|......|.....|..............
XP_075148       183 VEAHGTFFTNHCLGCRKLYSMEWMKDQIFSDNVPTCDDCNSVVKPDIVFFGENLPETFYS

XP_075153       423 ENGI

In [186]:
test_aligner2 = Align.PairwiseAligner(mode='local')
test_aligner2.open_gap_score = -10
test_aligner2.extend_gap_score = -0.5

ex_aln = test_aligner2.align(sequences[2], sequences[3])
print(ex_aln)
print(ex_aln[0])
print('='*80)
print(ex_aln[1])

<PairwiseAlignments object (6 alignments; score=70.5) at 0x7f6413015490>
XP_075153       183 PLNWVQKQIHSGVDPREILSKFLPPSIQRISPELTDMTLWRILASMMAEPPRRKKLSYVN
                  0 |.........................|..|....|.........................
XP_075148         4 PTSSNKNKSKDDNETSSSAQNEEDNTIEVIRKFFTQKLNLVTSLDEEDGANKKIFDNLTF

XP_075153       243 TFEDVIDLLKKSKNIIVLTGAGVSVSCGIPDFRSSDGIYSRLAKDFPNLPDPQAMFDINF
                 60 .........|..||.|...|||.|.|.||||||-|.|..........|||.|.|.|....
XP_075148        64 EGLCDHWKSKGFKNVITMVGAGISTSAGIPDFR-SPGSGLYDNLQKYNLPHPSAIFEMDY

XP_075153       303 FSKDPRPFYKFAREIYPGQFKPSPCHRFIKMLEQKQKLLRNYTQNIDTLEQVAGIKNVIE
                120 |...|.||...|.|.|||.|.|.|.|.|...|..|..|||.|||||||||..|||-----
XP_075148       123 FEENPQPFFALAKELYPGSFNPTPSHYFVRLLHEKGLLLRHYTQNIDTLERIAGI-----

XP_075153       363 CHGSFSTASCTKCKYKCDADTIRADIFAQRIPVCPQCQPNVEQSVDASQPVSEMELRMIV
                180 -----------------.||.............|..|...........|..|........
XP_075148       178 -------------

In [ ]:
# Freeze this: <PairwiseAlignments object (2 alignments; score=9) at 0x7f6413150b90>
test_aligner = Align.PairwiseAligner(mode='local')
test_aligner.match_score = 1.0
test_aligner.mismatch_score = -9.5
test_aligner.open_gap_score = -1.5
test_aligner.internal_gap_score=-2.5
ex_aln = test_aligner.align(sequences[2], sequences[3])
ex_aln
print(ex_aln)
print(ex_aln[0])
print('='*80)
print(ex_aln[1])

<PairwiseAlignments object (2 alignments; score=9) at 0x7f64126b77d0>

In [215]:
test_aligner2 = Align.PairwiseAligner(mode='global')
test_aligner2.open_gap_score = -7.1
# test_aligner2.extend_gap_score = -0.5
# test_aligner.internal_gap_score=-2.5

ex_aln = test_aligner2.align(sequences[2], sequences[3])
print(ex_aln)
x = np.array([aln.score for aln in ex_aln])
print(x.min(), x.max(), x.mean(), x.std())
# for aln in ex_aln:
#     print("Score = %.1f:" % aln.score)
# # print(ex_aln[0])
# print('='*80)
# print("Score = %.1f:" % ex_aln[1].score)
# # print(ex_aln[1])

<PairwiseAlignments object (1512 alignments; score=-407.5) at 0x7f6403c08ad0>
-407.5 -407.5 -407.5 0.0


In [249]:
test_aligner2 = Align.PairwiseAligner(mode='local')
test_aligner2.open_gap_score = -28
test_aligner2.extend_gap_score = -2.5
test_aligner2.internal_gap_score=-5.5
test_aligner2.match_score = .7


ex_aln = test_aligner2.align(sequences[2], sequences[3])
print(ex_aln)
for ialn, aln in enumerate(ex_aln):
    print(aln)
    if ialn>3: break
    print('='*80)


<PairwiseAlignments object (3 alignments; score=44.2) at 0x7f64140d69d0>
XP_075153       183 PLNWVQKQIHSGVDPREILSKFLPPSIQRISPELTDMTLWRILASMMAEPPRRKKLSYVN
                  0 |.........................|..|....|.........................
XP_075148         4 PTSSNKNKSKDDNETSSSAQNEEDNTIEVIRKFFTQKLNLVTSLDEEDGANKKIFDNLTF

XP_075153       243 TFEDVIDLLKKSKNIIVLTGAGVSVSCGIPDFRSSDGIYSRLAKDFPNLPDPQAMFDINF
                 60 .........|..||.|...|||.|.|.||||||-|.|..........|||.|.|.|....
XP_075148        64 EGLCDHWKSKGFKNVITMVGAGISTSAGIPDFR-SPGSGLYDNLQKYNLPHPSAIFEMDY

XP_075153       303 FSKDPRPFYKFAREIYPGQFKPSPCHRFIKMLEQKQKLLRNYTQNIDTLEQVAGIKNVIE
                120 |...|.||...|.|.|||.|.|.|.|.|...|..|..|||.|||||||||..|||.....
XP_075148       123 FEENPQPFFALAKELYPGSFNPTPSHYFVRLLHEKGLLLRHYTQNIDTLERIAGIPADKL

XP_075153       363 CHGSFSTASCTKCKYKCDADTIRADIFAQRIPVCPQCQPNVEQSVDASQPVSEMELRMIV
                180 ................................|......|.....|..............
XP_075148       183 VEAHGTFFTNHCL

In [ ]:
''' Main subsequence alignments from above cell: 
<PairwiseAlignments object (3 alignments; score=44.2) at 0x7f64140d7bd0>

XP_075153       243 TFEDVIDLLKKSKNIIVLTGAGVSVSCGIPDFRSSDGIYSRLAKDFPNLPDPQAMFDINF
                 60 .........|..||.|...|||.|.|.||||||-|.|..........|||.|.|.|....
XP_075148        64 EGLCDHWKSKGFKNVITMVGAGISTSAGIPDFR-SPGSGLYDNLQKYNLPHPSAIFEMDY

XP_075153       303 FSKDPRPFYKFAREIYPGQFKPSPCHRFIKMLEQKQKLLRNYTQNIDTLEQVAGIKNVIE
                120 |...|.||...|.|.|||.|.|.|.|.|...|..|..|||.|||||||||..|||.....
XP_075148       123 FEENPQPFFALAKELYPGSFNPTPSHYFVRLLHEKGLLLRHYTQNIDTLERIAGIPADKL

================================================================================

XP_075153       243 TFEDVIDLLKKSKNIIVLTGAGVSVSCGIPDFRSSDGIYSRLAKDFPNLPDPQAMFDINF
                 60 .........|..||.|...|||.|.|.|||||||-.|..........|||.|.|.|....
XP_075148        64 EGLCDHWKSKGFKNVITMVGAGISTSAGIPDFRS-PGSGLYDNLQKYNLPHPSAIFEMDY

XP_075153       303 FSKDPRPFYKFAREIYPGQFKPSPCHRFIKMLEQKQKLLRNYTQNIDTLEQVAGIKNVIE
                120 |...|.||...|.|.|||.|.|.|.|.|...|..|..|||.|||||||||..|||.....
XP_075148       123 FEENPQPFFALAKELYPGSFNPTPSHYFVRLLHEKGLLLRHYTQNIDTLERIAGIPADKL

================================================================================

XP_075153       243 TFEDVIDLLKKSKNIIVLTGAGVSVSCGIPDFRSSDGIYSRLAKDFPNLPDPQAMFDINF
                 60 .........|..||.|...|||.|.|.|||||||.-|..........|||.|.|.|....
XP_075148        64 EGLCDHWKSKGFKNVITMVGAGISTSAGIPDFRSP-GSGLYDNLQKYNLPHPSAIFEMDY

XP_075153       303 FSKDPRPFYKFAREIYPGQFKPSPCHRFIKMLEQKQKLLRNYTQNIDTLEQVAGIKNVIE
                120 |...|.||...|.|.|||.|.|.|.|.|...|..|..|||.|||||||||..|||.....
XP_075148       123 FEENPQPFFALAKELYPGSFNPTPSHYFVRLLHEKGLLLRHYTQNIDTLERIAGIPADKL

260->360
82->183

================================================================================
cf, different gap score param:

XP_075153       243 TFEDVIDLLKKSKNIIVLTGAGVSVSCGIPDFRSSDGIYSRLAKDFPNLPDPQAMFDINF
                 60 .........|..||.|...|||.|.|.|||||||.-|..........|||.|.|.|....
XP_075148        64 EGLCDHWKSKGFKNVITMVGAGISTSAGIPDFRSP-GSGLYDNLQKYNLPHPSAIFEMDY

XP_075153       303 FSKDPRPFYKFAREIYPGQFKPSPCHRFIKMLEQKQKLLRNYTQNIDTLEQVAGIKNVIE
                120 |...|.||...|.|.|||.|.|.|.|.|...|..|..|||.|||||||||..|||.....
XP_075148       123 FEENPQPFFALAKELYPGSFNPTPSHYFVRLLHEKGLLLRHYTQNIDTLERIAGIPADKL




So, a loose assert:
'''


In [ ]:
test_aligner = Align.PairwiseAligner(mode='local')
test_aligner.open_gap_score = -1
test_aligner.extend_gap_score = -2.5
test_aligner.internal_gap_score=-4.5
test_aligner.match_score = 1.0

ex_aln = test_aligner.align(sequences[1], sequences[4])
print(ex_aln)
print(ex_aln[0])

In [275]:
test_aligner2 = Align.PairwiseAligner(mode='local')
# test_aligner2.open_gap_score = -28
test_aligner2.extend_gap_score = -2.5
# test_aligner2.internal_gap_score=-5.5
# test_aligner2.match_score = .7

seq1 = sequences[2][262:390]
seq2 = sequences[3][82:193]

ex_aln = test_aligner2.align(seq1, seq2)
print(ex_aln)
for ialn, aln in enumerate(ex_aln):
    print(aln)
    if ialn>3: break
    print('='*80)


<PairwiseAlignments object (30 alignments; score=53) at 0x7f6411e16d90>
XP_075153         0 GAGVSVSCGIPDFRS-SDGIYSRLAKDFPNLPDPQAMFDINFFSKDPRPFYKFAREIYPG
                  0 |||.|.|.|||||||-..|.|..|.|-.-|||.|.|.|....|...|.||...|.|.|||
XP_075148         1 GAGISTSAGIPDFRSPGSGLYDNLQK-Y-NLPHPSAIFEMDYFEENPQPFFALAKELYPG

XP_075153        59 QFKPSPCHRFIKMLEQKQKLLRNYTQNIDTLEQVAGI-K-NVIECHGSFST 108
                 60 .|.|.|.|.|...|..|..|||.|||||||||..|||-.-...|.||.|.| 111
XP_075148        59 SFNPTPSHYFVRLLHEKGLLLRHYTQNIDTLERIAGIPADKLVEAHGTFFT 110

XP_075153         0 GAGVSVSCGIPDFRSS-DGIYSRLAKDFPNLPDPQAMFDINFFSKDPRPFYKFAREIYPG
                  0 |||.|.|.|||||||.-.|.|..|.|-.-|||.|.|.|....|...|.||...|.|.|||
XP_075148         1 GAGISTSAGIPDFRSPGSGLYDNLQK-Y-NLPHPSAIFEMDYFEENPQPFFALAKELYPG

XP_075153        59 QFKPSPCHRFIKMLEQKQKLLRNYTQNIDTLEQVAGI-K-NVIECHGSFST 108
                 60 .|.|.|.|.|...|..|..|||.|||||||||..|||-.-...|.||.|.| 111
XP_075148        59 SFNPTPSHYFVRLLHEKGLLLRHYTQNIDTLERIAGIPA

In [296]:
'''
Main sequences:
GAGaSbSCGIPDFRS
GIPDFRS
YTQNIDTLE
a: V or I  medium nonpolar
b: V or T  nonpolar
c: C or A  cysteine, alanine **
'''
for i in range(5):
    ind=sequences[i].seq.find('GAG')
    c_loc = sequences[i].seq[ind:ind+25].find('C')
    if c_loc>0:
        print(sequences[i].description)
        print(sequences[i].seq.find('GAG'))
        print(sequences[i].seq[ind:ind+25])
        print('^^^ C Cysteine found, at global position '+str(c_loc+ind))
        



XP_075153850.1 sirtuin 1 [Haematobia irritans]
262
GAGVSVSCGIPDFRSSDGIYSRLAK
^^^ C Cysteine found, at global position 269
AAI52315.1 Sirt1 protein [Mus musculus]
91
GAGVSVSCGIPDFRSRDGIYARLAV
^^^ C Cysteine found, at global position 98


In [304]:
test_aligner2 = Align.PairwiseAligner(mode='local')
# test_aligner2.open_gap_score = -28
# test_aligner2.extend_gap_score = -1.5
# test_aligner2.internal_gap_score=-5.5
# test_aligner2.match_score = .7

seq1 = sequences[2]
seq2 = sequences[4]

ex_aln = test_aligner2.align(seq1, seq2)
print(ex_aln)
for ialn, aln in enumerate(ex_aln):
    print(aln)
    if ialn>3: break
    print('='*80)

<PairwiseAlignments object (1287385983072000 alignments; score=228) at 0x7f6412697290>
XP_075153       179 ITQR-PLNWVQKQIHSGVDPREILSKFLPPSIQRISPELTDMTLWRILASMMAEPPRRKK
                  0 |..|-|...||.....|.|||.||...||..|..--|||.|||||.|......|||.|||
AAI52315.         9 IGYRGPYTFVQQHLMIGTDPRTILKDLLPETIPP--PELDDMTLWQIVINILSEPPKRKK

XP_075153       238 LSYVNTFEDVIDLLKKSKNIIVLTGAGVSVSCGIPDFRSSDGIYSRLAKDFPNLPDPQAM
                 60 ....||.||...||...|.||||||||||||||||||||.||||.|||.|||.|||||||
AAI52315.        67 RKDINTIEDAVKLLQECKKIIVLTGAGVSVSCGIPDFRSRDGIYARLAVDFPDLPDPQAM

XP_075153       298 FDINFFSKDPRPFYKFAREIYPGQFKPSPCHRFIKMLEQKQ-KLLRNYTQNIDTLEQVAG
                120 |||..|.||||||.|||.|||||||.||.||.||.-|..|.-||||||||||||||||||
AAI52315.       127 FDIEYFRKDPRPFFKFAKEIYPGQFQPSLCHKFIA-LSDKEGKLLRNYTQNIDTLEQVAG

XP_075153       357 IKNVIECHGSFSTASCTKCKYKCDADTIRADIFAQRIPVCPQCQPNVEQSVDASQPVSEM
                180 |.....|||||.||||..||||.|....|.|||.|..|.||.|-|..|-------|----
AAI52315.       186

In [305]:
'''New review of sirt1 mouse and sirt1 p aeruginosa.
XP_075153       179 ITQR-PLNWVQKQIHSGVDPREILSKFLPPSIQRISPELTDMTLWRILASMMAEPPRRKK
                  0 |..|-|...||.....|.|||.||...||..|.--.|||.|||||.|......|||.|||
AAI52315.         9 IGYRGPYTFVQQHLMIGTDPRTILKDLLPETIP--PPELDDMTLWQIVINILSEPPKRKK

XP_075153       238 LSYVNTFEDVIDLLKKSKNIIVLTGAGVSVSCGIPDFRSSDGIYSRLAKDFPNLPDPQAM
                 60 ....||.||...||...|.||||||||||||||||||||.||||.|||.|||.|||||||
AAI52315.        67 RKDINTIEDAVKLLQECKKIIVLTGAGVSVSCGIPDFRSRDGIYARLAVDFPDLPDPQAM

XP_075153       298 FDINFFSKDPRPFYKFAREIYPGQFKPSPCHRFIKMLEQKQ-KLLRNYTQNIDTLEQVAG
                120 |||..|.||||||.|||.|||||||.||.||.||.-|..|.-||||||||||||||||||
AAI52315.       127 FDIEYFRKDPRPFFKFAKEIYPGQFQPSLCHKFIA-LSDKEGKLLRNYTQNIDTLEQVAG

XP_075153       357 IKNVIECHGSFSTASCTKCKYKCDADTIRADIFAQRIPVCPQCQPNVEQSVDASQPVSEM
                180 |.....|||||.||||..||||.|....|.|||.|..|.||.|-|..|-------|----
AAI52315.       186 IQRILQCHGSFATASCLICKYKVDCEAVRGDIFNQVVPRCPRC-PADE-------P----

XP_075153       417 ELRMIVENGIMKPDIVFFGEGLPDEFHTVMGSDKDKCDLLIVMGSSLKVRPVALIPSSIP
                240 -|.-|-----|||.||||||.||..||..|..|||..|||||.|||||||||||||||||
AAI52315.       234 -LA-I-----MKPEIVFFGENLPEQFHRAMKYDKDEVDLLIVIGSSLKVRPVALIPSSIP

XP_075153       477 NNVPQILINREQLHHLEFDVELLGDGDVIINQICHRLGGDWTDICFDDEILQESNEFMSY
                300 ..|||||||||.|.||.||||||||.|||||..||||||.....|.....|.|..|....
AAI52315.       287 HEVPQILINREPLPHLHFDVELLGDCDVIINELCHRLGGEYAKLCCNPVKLSEITEKPPR


=> =>

'''
test_aligner2 = Align.PairwiseAligner(mode='local')

seq1 = sequences[2][179:537]
seq2 = sequences[4][:347]

ex_aln = test_aligner2.align(seq1, seq2)
print(ex_aln)
for ialn, aln in enumerate(ex_aln):
    print(aln)
    if ialn>3: break
    print('='*80)

<PairwiseAlignments object (22680 alignments; score=198) at 0x7f6411e9eb50>
XP_075153         0 ITQR-PLNWVQKQIHSGVDPREILSKFLPPSIQRISPELTDMTLWRILASMMAEPPRRKK
                  0 |..|-|...||.....|.|||.||...||..|..--|||.|||||.|......|||.|||
AAI52315.         9 IGYRGPYTFVQQHLMIGTDPRTILKDLLPETIPP--PELDDMTLWQIVINILSEPPKRKK

XP_075153        59 LSYVNTFEDVIDLLKKSKNIIVLTGAGVSVSCGIPDFRSSDGIYSRLAKDFPNLPDPQAM
                 60 ....||.||...||...|.||||||||||||||||||||.||||.|||.|||.|||||||
AAI52315.        67 RKDINTIEDAVKLLQECKKIIVLTGAGVSVSCGIPDFRSRDGIYARLAVDFPDLPDPQAM

XP_075153       119 FDINFFSKDPRPFYKFAREIYPGQFKPSPCHRFIKMLEQKQ-KLLRNYTQNIDTLEQVAG
                120 |||..|.||||||.|||.|||||||.||.||.||.-|..|.-||||||||||||||||||
AAI52315.       127 FDIEYFRKDPRPFFKFAKEIYPGQFQPSLCHKFIA-LSDKEGKLLRNYTQNIDTLEQVAG

XP_075153       178 IKNVIECHGSFSTASCTKCKYKCDADTIRADIFAQRIPVCPQCQPNVEQSVDASQPVSEM
                180 |.....|||||.||||..||||.|....|.|||.|..|.||.|-|..|-------|----
AAI52315.       186 IQRILQCHGS

In [307]:
seq1.description, seq2.description

('XP_075153850.1 sirtuin 1 [Haematobia irritans]',
 'AAI52315.1 Sirt1 protein [Mus musculus]')

In [ ]:
'''
('XP_075153850.1 sirtuin 1 [Haematobia irritans]',
 'AAI52315.1 Sirt1 protein [Mus musculus]')
 Offsets:  [179:537]  [:347]

XP_075153         0 ITQR-PLNWVQKQIHSGVDPREILSKFLPPSIQRISPELTDMTLWRILASMMAEPPRRKK
                  0 |..|-|...||.....|.|||.||...||..|.--.|||.|||||.|......|||.|||
AAI52315.         9 IGYRGPYTFVQQHLMIGTDPRTILKDLLPETIP--PPELDDMTLWQIVINILSEPPKRKK

XP_075153        59 LSYVNTFEDVIDLLKKSKNIIVLTGAGVSVSCGIPDFRSSDGIYSRLAKDFPNLPDPQAM
                 60 ....||.||...||...|.||||||||||||||||||||.||||.|||.|||.|||||||
AAI52315.        67 RKDINTIEDAVKLLQECKKIIVLTGAGVSVSCGIPDFRSRDGIYARLAVDFPDLPDPQAM

XP_075153       119 FDINFFSKDPRPFYKFAREIYPGQFKPSPCHRFIKMLEQKQ-KLLRNYTQNIDTLEQVAG
                120 |||..|.||||||.|||.|||||||.||.||.||.-|..|.-||||||||||||||||||
AAI52315.       127 FDIEYFRKDPRPFFKFAKEIYPGQFQPSLCHKFIA-LSDKEGKLLRNYTQNIDTLEQVAG

XP_075153       178 IKNVIECHGSFSTASCTKCKYKCDADTIRADIFAQRIPVCPQCQPNVEQSVDASQPVSEM
                180 |.....|||||.||||..||||.|....|.|||.|..|.||.|-|..|-------|----
AAI52315.       186 IQRILQCHGSFATASCLICKYKVDCEAVRGDIFNQVVPRCPRC-PADE-------P----

XP_075153       238 ELRMIVENGIMKPDIVFFGEGLPDEFHTVMGSDKDKCDLLIVMGSSLKVRPVALIPSSIP
                240 -|.-|-----|||.||||||.||..||..|..|||..|||||.|||||||||||||||||
AAI52315.       234 -LA-I-----MKPEIVFFGENLPEQFHRAMKYDKDEVDLLIVIGSSLKVRPVALIPSSIP

XP_075153       298 NNVPQILINREQLHHLEFDVELLGDGDVIINQICHRLGGDWTDICFDDEILQESNE 354
                300 ..|||||||||.|.||.||||||||.|||||..||||||.....|.....|.|..| 356
AAI52315.       287 HEVPQILINREPLPHLHFDVELLGDCDVIINELCHRLGGEYAKLCCNPVKLSEITE 343

Features. Horn fly sirtuin 1: right at 77, IIVL..., is where the sirtuin protein itself is coded; ie, from global 256->510 (local 331). Before that, from <236..262 is a protein of unknown function ( /region_name="DUF592"  /note="Protein of unknown function (DUF592); pfam04574"  /db_xref="CDD:368003" ).  
binding site is order(349,365,433,435..439,465..467) (local, [170, 186, 254, 256..260, 286..288])

'''
